## IMPORTING LIBRARIES

In [408]:
import numpy as np
import pandas as pd
import re
# from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn .model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report

## DATA PREPROCESSING

In [356]:
true_news=pd.read_csv("True.csv")

In [357]:
true_news.shape

(21417, 4)

In [358]:
fake_news=pd.read_csv("Fake.csv")

In [359]:
fake_news.shape

(23481, 4)

In [360]:
true_df = pd.concat([true_news[['title', 'text', 'subject', 'date']], pd.Series(0, index=true_news.index, name="label")], axis=1)
true_df

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [361]:
fake_df=pd.concat([fake_news[['title', 'text', 'subject', 'date']],pd.Series(1,index=fake_news.index,name="label")],axis=1)
fake_df

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [362]:
news_data=pd.concat([true_df,fake_df],axis=0).sample(frac=1.0,random_state=34).reset_index(drop=True)#.sample(frac=1.0) shuffles the entire DataFrame
news_data

,title,text,subject,date,label
0,SUNDAY SCREENING: ‘The War on Democracy’ (2007),"21st Century Wire says Each week, 21WIRE will ...",US_News,"August 13, 2017",1
1,SAY WHAT? Obama Gives Go Ahead For New UN “Reg...,"If seven years ago, someone told Americans tha...",left-news,"Nov 20, 2015",1
2,U.S. farm groups oppose Trump's cuts to agricu...,CHICAGO (Reuters) - U.S. farm groups on Tuesda...,politicsNews,"May 23, 2017",0
3,BREAKING: Federal Judge STOPS Obamacare Transg...,O Connor found that the plaintiffs had standin...,left-news,"Dec 31, 2016",1
4,OBAMA NOMINATES FIRST MUSLIM FOR FEDERAL BENCH...,President Obama is clearly not quite ready to ...,politics,"Sep 8, 2016",1
...,...,...,...,...,...
44893,Laura Bush Stuns Republican Party – Hinted Th...,It s no secret Republican women hate Donald Tr...,News,"April 10, 2016",1
44894,"Here’s How Much Trump Is Paying Omarosa, Kell...","The average American makes about $50,000 a yea...",News,"June 30, 2017",1
44895,REALLY FAKE NEWS: New York Times Finally Retra...,21st Century Wire says Back during the 2016 el...,US_News,"June 30, 2017",1
44896,WHOA! SENATOR TOM COTTON Blasts Harry Reid’s “...,Senator Tom Cotton is such a great representat...,Government News,"May 26, 2016",1


In [363]:
news_data.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [364]:
# merging the title and subject
news_data['content']=news_data['subject']+' '+news_data['title']

In [365]:
print(news_data['content'])

0        US_News SUNDAY SCREENING: ‘The War on Democrac...
1        left-news SAY WHAT? Obama Gives Go Ahead For N...
2        politicsNews U.S. farm groups oppose Trump's c...
3        left-news BREAKING: Federal Judge STOPS Obamac...
4        politics OBAMA NOMINATES FIRST MUSLIM FOR FEDE...
                               ...                        
44893    News  Laura Bush Stuns Republican Party – Hint...
44894    News  Here’s How Much Trump Is Paying Omarosa,...
44895    US_News REALLY FAKE NEWS: New York Times Final...
44896    Government News WHOA! SENATOR TOM COTTON Blast...
44897    worldnews UK's May to visit China around Jan. ...
Name: content, Length: 44898, dtype: object


In [366]:
# # separating the data and label
# x=news_data.drop(columns='label',axis=1)
# x

In [367]:
# y=news_data['label']
# y

In [368]:
#stemming
ps=PorterStemmer()

In [369]:
def process_content(content):
    new_content=content.lower()
    new_content=re.sub(r'\$[^\s]+','dollar',new_content)
    new_content=re.sub(r'[^a-z0-9\s]','',new_content)
    new_content=re.sub(r'[0-9]+','number',new_content)
    new_content=new_content.split(" ")
    new_content = list(map(lambda x: ps.stem(x), new_content))
    new_content=list(map(lambda x:x.strip(),new_content))
    if ''in new_content:
        new_content.remove("")
    return new_content

    

In [370]:
contents=news_data['content'].apply(process_content)

In [371]:
contents

0        [usnew, sunday, screen, the, war, on, democrac...
1        [leftnew, say, what, obama, give, go, ahead, f...
2        [politicsnew, us, farm, group, oppos, trump, c...
3        [leftnew, break, feder, judg, stop, obamacar, ...
4        [polit, obama, nomin, first, muslim, for, fede...
                               ...                        
44893    [news, laura, bush, stun, republican, parti, ,...
44894    [news, here, how, much, trump, is, pay, omaros...
44895    [usnew, realli, fake, news, new, york, time, f...
44896    [govern, news, whoa, senat, tom, cotton, blast...
44897    [worldnew, uk, may, to, visit, china, around, ...
Name: content, Length: 44898, dtype: object

In [372]:
# separating the data and labels
x=news_data['content'].values
x

array(['US_News SUNDAY SCREENING: ‘The War on Democracy’ (2007)',
       'left-news SAY WHAT? Obama Gives Go Ahead For New UN “Regional Hub” In Washington DC…What They Plan To Use Center For Is Disturbing',
       "politicsNews U.S. farm groups oppose Trump's cuts to agriculture spending",
       ...,
       'US_News REALLY FAKE NEWS: New York Times Finally Retracts Its ’17 Intelligence Agencies’ Claim on Russia Hacking US Elections',
       'Government News WHOA! SENATOR TOM COTTON Blasts Harry Reid’s “Cancerous Leadership” [Video]',
       "worldnews UK's May to visit China around Jan. 31: Sky News"],
      dtype=object)

In [373]:
y=news_data['label'].values
y

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [374]:
y.shape

(44898,)

In [375]:
# converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [376]:
print(x)

  (0, 166)	0.5366960689719623
  (0, 5224)	0.41123155570315506
  (0, 13010)	0.1564690850205703
  (0, 16429)	0.4354312285775774
  (0, 18123)	0.3788441382828912
  (0, 18676)	0.16390368242613051
  (0, 19799)	0.27308483634215946
  (0, 20215)	0.28948295578855127
  (1, 829)	0.21895825036201896
  (1, 3321)	0.25708337524042496
  (1, 4948)	0.2466341299677107
  (1, 5825)	0.2621216242319139
  (1, 7663)	0.1960863503106476
  (1, 8199)	0.20562466979624444
  (1, 8260)	0.20030560906277875
  (1, 9269)	0.3124698547990613
  (1, 9597)	0.09108409873259107
  (1, 10061)	0.12726360787140334
  (1, 10848)	0.11306684862176522
  (1, 12603)	0.14718843990780123
  (1, 12612)	0.07156524277533463
  (1, 12842)	0.12668962685483912
  (1, 13909)	0.18836906969428524
  (1, 15221)	0.27138949984464583
  (1, 16271)	0.195376131445715
  :	:
  (44895, 19799)	0.18477608476577587
  (44895, 20808)	0.2314605048127331
  (44896, 2323)	0.2694689475085966
  (44896, 3090)	0.4423757977136681
  (44896, 4471)	0.37816965959495846
  (44896, 833

## TRAINING AND TESTING

In [379]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,stratify=y,random_state=20)

In [381]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [383]:
## EVALUATION

,Actual y,Predicted y
0,0,0
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
8975,0,0
8976,1,1
8977,0,0
8978,1,1


In [394]:
y_train_pred=model.predict(x_train)
accuracy_score(y_train_pred,y_train)

0.9999443176123393

In [398]:
y_pred=model.predict(x_test)

In [400]:
pd.DataFrame({'Actual y':y_test,'Predicted y':y_pred})

,Actual y,Predicted y
0,0,0
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
8975,0,0
8976,1,1
8977,0,0
8978,1,1


In [404]:
accuracy_score(y_pred,y_test)

0.9997772828507795

In [412]:
c=classification_report(y_pred,y_test)
print('Classification Report',c)

Classification Report               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4282
           1       1.00      1.00      1.00      4698

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



##### Making a predictive system

In [417]:
x_new=x_test[1]
prediction=model.predict(x_new)
print(prediction)

if (prediction[0]==0):
    print('The news is Real')
else:
    print('The news is fake')

[1]
The news is fake
